In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
import joblib
import time
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

### Data voorbereiden
Ik laad hier de studenten set, die de trancripts van Ted video's heeft

In [2]:
file_path = '/mnt/data/studentset.csv'
data = pd.read_csv("./studentset.csv")

Ik deel de data nu op in x voor de transcripts, en y voor de labels (dus het resultaat)

In [3]:
X = data['transcript']
y = data['label']

### Model Trainen
Ik ga 20% van mijn data gebruiken als test set, om mijn model te kunnen controleren met data waarop hij niet getrained is

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Ik vecorize de transcrips en haal de stop woorden uit de data, zodat mijn AI beter getrained kan worden

In [5]:
count_vectorizer = CountVectorizer(stop_words='english', min_df=6, max_df=0.90, binary=True)
X_train_bow = count_vectorizer.fit_transform(X_train)
X_test_bow = count_vectorizer.transform(X_test)

Ik gebruik hier de Native Bayes classifier om de freqeuntie van woorden te kunnen meten en dat gebruik ik dan om mijn ai verder op te trainen

In [6]:
clf = MultinomialNB()
clf.fit(X_train_bow, y_train)

MultinomialNB()

### Model Testen
Ik maak nu de predictie met mijn getrainde model op de training set

In [7]:
predicted = clf.predict(X_test_bow)

Hier beoordeel ik de accuratie van mijn AI, het is 0.659 (65.9%) wat niet super is maar het is wel een stuk boven de gok kans. Dit kan bijvoorbeeld liggen aan de kwaliteit van de data of de complexiteit van de tekst.

In [8]:
accuracy = accuracy_score(y_test, predicted)
print(f'Accuracy: {accuracy}')

Accuracy: 0.6594594594594595


Dit is mijn confusion matrix, ik heb dus 107 True negative, 137 True positive, 47 False negative en 79 False positive. Dit komt neer op dat ik 69% van de negatieve correct heb beoordeeld en 63% van de positieve correct heb beoordeeld

In [9]:
cm = confusion_matrix(y_test, predicted)
print('Confusion Matrix:')
print(cm)

Confusion Matrix:
[[107  79]
 [ 47 137]]


Mijn precision score is 0.634, dit betekent dat 63.4% van de positieve voorspellingen correct is.

In [10]:
ps = precision_score(y_test, predicted)
print(ps)

0.6342592592592593


Mijn recall score is 0.744, dit betekent dat 74.4% van de werkelijke positieve gevallen correct is geclassificeerd.

In [11]:
rs = recall_score(y_test, predicted) 
print(rs)

0.7445652173913043


Mijn f1 score is 0.657, de F1-score is de harmonische gemiddelde van precision en recall. Dit betekent dat het model redelijk goed presteert over beide metrics, maar er is ruimte voor verbetering.

In [12]:
f1 = f1_score(y_test, predicted, average='weighted')  
f1

0.6570556438791734

Mijn Throughput is 7.1431 inferenties per miliseconden, dus hij doet een predictie in ~7 milisecondes, dit is vergelijkbaar wat ik zie bij klasgenoten

In [13]:
# Bepaal het aantal inferenties voor throughput-berekening
n_inferences = 100 

# Starttijd voor throughput-meting
start_ts = time.time()

# Voer meerdere inferenties uit voor de throughput-berekening
for i in range(n_inferences):
    _ = clf.predict(X_test_bow[i:i+1].toarray())

# Eindtijd na alle inferenties
end_ts = time.time()

# Totale tijd voor de batch inferenties in milliseconden
total_time_ms = (end_ts - start_ts) * 1000

# Bereken throughput (aantal inferenties per milliseconde)
throughput_ms = n_inferences / total_time_ms
print(f"Throughput: {throughput_ms:.4f} inferenties per milliseconde")

Throughput: 7.1431 inferenties per milliseconde


Mijn latency is 0.0194 seconden, ongeveer 19.4 milliseconden per inferentie, dit is een stuk beter dan de benchmark van school. latency is de tijd die het model nodig heeft om een enkele voorspelling te doen.

In [14]:
end_ts = time.time()
latency = end_ts - start_ts
print(f"Latency voor één inferentie: {latency:.4f} seconden")

Latency voor één inferentie: 0.0194 seconden


### Model Opslaan
Hier sla ik mijn model en de vectorizer op in een joblib file om in mijn deployment te gebruiken

In [15]:
model_filename = 'sentiment_model.joblib'
vectorizer_filename = 'vectorizer.joblib'

joblib.dump(clf, model_filename)

joblib.dump(count_vectorizer, vectorizer_filename)

['vectorizer.joblib']